In [3]:
import pandas as pd
import numpy as np
import warnings
import time
import itertools
import copy
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)

In [4]:
dataset_name = "News"

# 1. Read  Data

In [5]:
MIND_dev_news_df = pd.read_csv("../Dataset/MINDsmall_dev/news.tsv", header=None, delimiter="\t")
MIND_dev_news_df.columns = ["newsid","category","subcategory","title","abstract","url","title_entities","abstract_entities"]
MIND_dev_news_df.drop(['url','title_entities','abstract_entities'], axis=1, inplace=True)
MIND_dev_news_df
(MIND_dev_news_df.isna().sum()/MIND_dev_news_df.shape[0]).sort_values(ascending=False)

,newsid,category,subcategory,title,abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without."
1,N18955,health,medical,Dispose of unwanted prescription drugs during the DEA's Take Back Day,NaN
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge..."
...,...,...,...,...,...
42411,N63550,lifestyle,lifestyleroyals,Why Kate & Meghan Were on Different Balconies for Remembrance Sunday,There's no scandal here. It's all about the order of precedence.
42412,N30345,entertainment,entertainment-celebrity,See the stars at the 2019 Baby2Baby gala,Stars like Chrissy Teigen and Kate Hudson support Baby2Baby at their annual gala
42413,N30135,news,newsgoodnews,Tennessee judge holds lawyer's baby as he swears her into the state bar in viral video,Tennessee Court of Appeals Judge Richard Dinkins is going viral after a video showed him swearin...
42414,N44276,autos,autossports,Best Sports Car Deals for October,NaN


abstract       0.047647
newsid         0.000000
category       0.000000
subcategory    0.000000
title          0.000000
dtype: float64

In [6]:
MIND_dev_news_df_fliter = MIND_dev_news_df.dropna().reset_index(drop=True)
MIND_dev_news_df_fliter

,newsid,category,subcategory,title,abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without."
1,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...
2,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
3,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge..."
4,N2073,sports,football_nfl,Should NFL be able to fine players for criticizing officiating?,Several fines came down against NFL players for criticizing officiating this week. It's a very b...
...,...,...,...,...,...
40390,N42491,movies,movies-celebrity,Roman Polanski Denies Rape Allegation by Valentine Monnier,"Responding to French actress Valentine Monnier's accusation of rape, a lawyer for Roman Polanski..."
40391,N13097,movies,movienews,"Marvel's Kevin Feige Breaks Silence on Scorsese Attack: ""It's Unfortunate"" (Exclusive)","In his first public comments about the debate raging over whether or not his films are cinema, t..."
40392,N63550,lifestyle,lifestyleroyals,Why Kate & Meghan Were on Different Balconies for Remembrance Sunday,There's no scandal here. It's all about the order of precedence.
40393,N30345,entertainment,entertainment-celebrity,See the stars at the 2019 Baby2Baby gala,Stars like Chrissy Teigen and Kate Hudson support Baby2Baby at their annual gala


In [7]:
drop_newsid_list = pd.concat([MIND_dev_news_df, MIND_dev_news_df_fliter]).drop_duplicates(["newsid"],keep=False).newsid.values.tolist()
drop_newsid_list

['N18955',
 'N26094',
 'N41835',
 'N8105',
 'N43944',
 'N45191',
 'N31161',
 'N36464',
 'N58035',
 'N16077',
 'N18885',
 'N5682',
 'N46832',
 'N4021',
 'N12987',
 'N20818',
 'N42607',
 'N39535',
 'N22256',
 'N61483',
 'N59137',
 'N26634',
 'N55663',
 'N33853',
 'N14028',
 'N41357',
 'N8263',
 'N33800',
 'N26401',
 'N29960',
 'N61433',
 'N50652',
 'N37046',
 'N37694',
 'N52571',
 'N34777',
 'N6351',
 'N26460',
 'N50438',
 'N47326',
 'N28119',
 'N27395',
 'N42785',
 'N31524',
 'N47487',
 'N53734',
 'N59740',
 'N16248',
 'N35667',
 'N58928',
 'N32770',
 'N28760',
 'N53006',
 'N2025',
 'N13468',
 'N17061',
 'N51559',
 'N57488',
 'N33146',
 'N47729',
 'N13946',
 'N20823',
 'N24187',
 'N44131',
 'N59010',
 'N41373',
 'N58452',
 'N55422',
 'N59332',
 'N62655',
 'N36321',
 'N20115',
 'N29148',
 'N9029',
 'N15212',
 'N17344',
 'N1466',
 'N10917',
 'N9427',
 'N7208',
 'N62969',
 'N61588',
 'N30450',
 'N46975',
 'N58742',
 'N54359',
 'N55900',
 'N46346',
 'N37362',
 'N43421',
 'N40272',
 'N17314'

In [8]:
MIND_dev_df = pd.read_csv("../Dataset/MINDsmall_dev/behaviors.tsv", header=None, delimiter="\t")
MIND_dev_df.columns = ["impressionid","userid","time","history","impression"]
MIND_dev_df["history"] = MIND_dev_df["history"].apply(lambda x: str(x).split(' '))
MIND_dev_df["impression"] = MIND_dev_df["impression"].apply(lambda x: str(x).split(' '))
MIND_dev_df

,impressionid,userid,time,history,impression
0,1,U80234,11/15/2019 12:37:50 PM,"[N55189, N46039, N51741, N53234, N11276, N264, N40716, N28088, N43955, N6616, N47686, N63573, N3...","[N28682-0, N48740-0, N31958-1, N34130-0, N6916-0, N5472-0, N50775-0, N24802-0, N19990-0, N33176-..."
1,2,U60458,11/15/2019 7:11:50 AM,"[N58715, N32109, N51180, N33438, N54827, N28488, N61186, N34775, N33742, N50020, N57061, N30924,...","[N20036-0, N23513-1, N32536-0, N46976-0, N35216-0, N36779-0, N31958-0]"
2,3,U44190,11/15/2019 9:55:12 AM,"[N56253, N1150, N55189, N16233, N61704, N51706, N53033, N15634, N3259]","[N36779-0, N62365-0, N58098-0, N5472-0, N13408-0, N55036-0, N19990-0, N53283-0, N20036-0, N47383..."
3,4,U87380,11/15/2019 3:12:46 PM,"[N63554, N49153, N28678, N23232, N43369, N58518, N44402, N7649, N63429, N45794, N53531, N53033, ...","[N6950-0, N60215-0, N6074-0, N11930-0, N6916-0, N24802-0, N48740-0, N60675-0, N45057-0, N51470-0..."
4,5,U9444,11/15/2019 8:25:46 AM,"[N51692, N18285, N26015, N22679, N55556]","[N5940-1, N23513-0, N49285-0, N23355-0, N19990-0, N31958-1, N29393-0, N30290-0, N19611-0, N62365..."
...,...,...,...,...,...
73147,73148,U77536,11/15/2019 8:40:16 PM,"[N28691, N8845, N58434, N37120, N22185, N60033, N47020, N60340, N46886, N7884, N8224, N46259, N4...","[N496-0, N35159-0, N59856-0, N13270-0, N47213-0, N26485-0, N50565-0, N10083-0, N60762-0, N38324-..."
73148,73149,U56193,11/15/2019 1:11:26 PM,"[N4705, N58782, N53531, N46492, N26026, N28088, N31099, N38311, N28257]","[N49285-0, N31958-0, N55237-0, N42844-0, N29862-0, N19990-0, N51470-0, N24802-0, N62365-0, N6400..."
73149,73150,U16799,11/15/2019 3:37:06 PM,"[N40826, N42078, N15670, N15295, N64536, N46845, N52294]","[N7043-0, N512-0, N60215-1, N45057-0, N496-0, N37055-0, N16344-0, N53283-0, N6194-0, N16120-0, N..."
73150,73151,U8786,11/15/2019 8:29:26 AM,"[N3046, N356, N20483, N46107, N44598, N18693, N8254, N51299, N28088, N18870]","[N23692-0, N19990-0, N20187-0, N5940-0, N13408-0, N31958-0, N30290-0, N42844-0, N6916-0, N58098-..."


# 2. Fliter

In [9]:
def get_clean_impression_and_history(row):
    impression_list = []
    impression_pos_list = []
    label_list = []
    flag = 0
    for i in row.impression:
        impression, label = i.split("-")
        if impression in drop_newsid_list:
            continue
        label = int(label)
        if label == 1:
            impression_pos_list.append(impression)
            if flag == 1:
                continue
            else:
                impression_list.append(impression)
                label_list.append(label)
                flag=1
        else:
            impression_list.append(impression)
            label_list.append(label)
    
    if 1 not in label_list:
        pos_target = -1
    else:
        pos_target = impression_list[label_list.index(1)]
        
    history_list = copy.deepcopy(row.history)
    for i in row.history:
        if i in drop_newsid_list:
            history_list.remove(i)
     
    return [impression_list, label_list, pos_target, history_list, impression_pos_list]

In [10]:
MIND_dev_df[["impression_list", "label_list", "pos_target", "history_list", "impression_pos_list"]] = MIND_dev_df.apply(get_clean_impression_and_history, axis=1, result_type="expand")
MIND_dev_df.drop(['time','impression','history'], axis=1, inplace=True)
MIND_dev_df

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list
0,1,U80234,"[N28682, N48740, N31958, N34130, N6916, N5472, N50775, N24802, N19990, N33176, N62365, N5940, N6...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N31958,"[N55189, N46039, N51741, N53234, N11276, N264, N40716, N28088, N43955, N6616, N47686, N63573, N3...",[N31958]
1,2,U60458,"[N20036, N32536, N46976, N35216, N36779, N31958]","[0, 0, 0, 0, 0, 0]",-1,"[N58715, N32109, N51180, N33438, N54827, N28488, N61186, N34775, N33742, N50020, N57061, N30924,...",[]
2,3,U44190,"[N36779, N62365, N58098, N5472, N13408, N55036, N19990, N53283, N20036, N47383, N37352, N31958, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",N5940,"[N56253, N1150, N55189, N16233, N61704, N51706, N53033, N15634, N3259]",[N5940]
3,4,U87380,"[N6950, N60215, N6074, N11930, N6916, N24802, N48740, N60675, N45057, N51470, N62365, N15347, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N15347,"[N63554, N49153, N28678, N43369, N58518, N44402, N7649, N63429, N45794, N53531, N53033, N30765, ...",[N15347]
4,5,U9444,"[N5940, N49285, N23355, N19990, N29393, N30290, N19611, N62365, N51470, N34130, N36779, N20036]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N5940,"[N51692, N18285, N26015, N22679, N55556]","[N5940, N31958]"
...,...,...,...,...,...,...,...
73147,73148,U77536,"[N496, N35159, N59856, N13270, N47213, N26485, N50565, N10083, N60762, N38324, N46162, N4238, N6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N38324,"[N28691, N8845, N37120, N22185, N60033, N47020, N60340, N46886, N7884, N8224, N46259, N46301, N2...","[N38324, N6916, N56080, N54562, N13573, N60244, N64228, N56969, N3168]"
73148,73149,U56193,"[N49285, N31958, N55237, N42844, N29862, N19990, N51470, N24802, N62365, N6400, N11390, N48740, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",N11390,"[N4705, N58782, N53531, N46492, N26026, N28088, N31099, N38311, N28257]",[N11390]
73149,73150,U16799,"[N7043, N512, N60215, N45057, N496, N37055, N16344, N53283, N6194, N16120, N26488, N12409, N5503...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N60215,"[N40826, N42078, N15670, N15295, N64536, N46845, N52294]","[N60215, N54562]"
73150,73151,U8786,"[N23692, N19990, N20187, N5940, N13408, N31958, N30290, N42844, N6916, N58098, N29393, N36940, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",N20036,"[N3046, N356, N20483, N46107, N44598, N18693, N8254, N51299, N28088, N18870]",[N20036]


In [11]:
impression_list = MIND_dev_df["impression_pos_list"].values.tolist()
history_list = MIND_dev_df["history_list"].values.tolist()
impression_list = list(itertools.chain(*impression_list))
history_list = list(itertools.chain(*history_list))
impression_list = pd.Series(impression_list + history_list)
item_pop = impression_list.value_counts().rename_axis("itemid").reset_index(name="pop")
item_pop

,itemid,pop
0,N42620,10116
1,N306,9733
2,N31958,8042
3,N45794,7678
4,N43142,6971
...,...,...
37876,N30869,1
37877,N24776,1
37878,N6223,1
37879,N15845,1


In [12]:
MIND_dev_df = MIND_dev_df[MIND_dev_df["pos_target"]!=-1].reset_index(drop=True)
MIND_dev_df["history_len"] = MIND_dev_df["history_list"].apply(lambda x: len(x))
MIND_dev_df["impression_len"] = MIND_dev_df["impression_list"].apply(lambda x: len(x))
MIND_dev_df_fliter = MIND_dev_df[(MIND_dev_df["history_len"]>=10) & (MIND_dev_df["impression_len"]>=25)].reset_index(drop=True)
MIND_dev_df_fliter

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len
0,4,U87380,"[N6950, N60215, N6074, N11930, N6916, N24802, N48740, N60675, N45057, N51470, N62365, N15347, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N15347,"[N63554, N49153, N28678, N43369, N58518, N44402, N7649, N63429, N45794, N53531, N53033, N30765, ...",[N15347],17,26
1,11,U24918,"[N54125, N60799, N29862, N3159, N46749, N40982, N37793, N31958, N6400, N1952, N38620, N7419, N10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N24802,"[N10792, N63241, N26424, N49745, N46978, N6233, N61103, N12907, N10865, N2783, N55743, N63554, N...","[N24802, N15347]",23,91
2,12,U50227,"[N49285, N38951, N62365, N28682, N5472, N26572, N42844, N31958, N12446, N6950, N11390, N11930, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N11390,"[N30974, N7171, N22561, N51630, N55951, N12098, N33922, N45794, N17559, N36636, N49792, N52551, ...","[N11390, N51470, N24802]",35,37
3,14,U47134,"[N49554, N12446, N7342, N6638, N12320, N10423, N5940, N60747, N2852, N61829, N37204, N50894, N37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N36940,"[N35809, N13395, N23284, N64496, N29177, N1150, N26136, N37812, N10629, N10059, N27676, N10343, ...",[N36940],56,124
4,16,U18529,"[N16118, N28072, N37204, N6645, N57515, N15052, N15817, N60939, N29490, N19643, N32536, N5981, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N13854,"[N28850, N17770, N33458, N13724, N12760, N51396, N53033, N501, N64425, N64124, N56148, N16715, N...","[N13854, N5507]",17,117
...,...,...,...,...,...,...,...,...,...
24185,73142,U37385,"[N38915, N62992, N13347, N10051, N61197, N48487, N32567, N54752, N38311, N7419, N19643, N13408, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N44249,"[N61471, N3046, N59173, N19048, N42526, N9226, N9120, N29898, N5102, N61708, N306, N64273, N14006]",[N44249],13,160
24186,73145,U92886,"[N59933, N1952, N60724, N55036, N44621, N21171, N56969, N48740, N19990, N6950, N7556, N11390, N2...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N1952,"[N28741, N35450, N31801, N8201, N14617, N5742, N18656, N64531, N29135, N34862, N34040, N306, N12...","[N1952, N6400, N45319]",24,40
24187,73146,U23543,"[N59602, N19990, N53572, N5472, N27057, N36779, N55237, N5940, N46976, N6916, N50775, N47383, N5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",N36940,"[N22570, N19638, N47525, N17210, N22161, N28058, N46994, N16715, N1637, N370, N51015, N27863, N1...",[N36940],19,26
24188,73147,U67440,"[N11930, N50775, N33439, N7556, N3972, N42844, N36779, N46162, N12446, N51470, N19990, N62658, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N48426,"[N55312, N9897, N55846, N1569, N6956, N28501, N11641, N15542, N13304, N9612, N4830, N38621, N283...",[N48426],73,41


In [13]:
def truncat_data(row, n_candidate=10, n_history=10):
    # trunct candiadte
    cache = copy.deepcopy(row.impression_list)
    cache.remove(row.pos_target)
    neg_target = cache.pop(0)
    candidate = [row.pos_target] + [neg_target] + cache[:n_candidate-2]
    # trunct history
    history = row.history_list[-n_history:]
    
    return neg_target, candidate, history

MIND_dev_df_fliter[["neg_target", "full_candidate", "full_history"]] = MIND_dev_df_fliter.apply(truncat_data, n_candidate=25, n_history=10, axis=1, result_type="expand")
MIND_dev_df_fliter.shape[0]
MIND_dev_df_fliter

24190

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len,neg_target,full_candidate,full_history
0,4,U87380,"[N6950, N60215, N6074, N11930, N6916, N24802, N48740, N60675, N45057, N51470, N62365, N15347, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N15347,"[N63554, N49153, N28678, N43369, N58518, N44402, N7649, N63429, N45794, N53531, N53033, N30765, ...",[N15347],17,26,N6950,"[N15347, N6950, N60215, N6074, N11930, N6916, N24802, N48740, N60675, N45057, N51470, N62365, N2...","[N63429, N45794, N53531, N53033, N30765, N34452, N24298, N29361, N2597, N28247]"
1,11,U24918,"[N54125, N60799, N29862, N3159, N46749, N40982, N37793, N31958, N6400, N1952, N38620, N7419, N10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N24802,"[N10792, N63241, N26424, N49745, N46978, N6233, N61103, N12907, N10865, N2783, N55743, N63554, N...","[N24802, N15347]",23,91,N54125,"[N24802, N54125, N60799, N29862, N3159, N46749, N40982, N37793, N31958, N6400, N1952, N38620, N7...","[N42620, N3899, N23124, N18916, N47686, N55911, N20397, N601, N48742, N46845]"
2,12,U50227,"[N49285, N38951, N62365, N28682, N5472, N26572, N42844, N31958, N12446, N6950, N11390, N11930, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N11390,"[N30974, N7171, N22561, N51630, N55951, N12098, N33922, N45794, N17559, N36636, N49792, N52551, ...","[N11390, N51470, N24802]",35,37,N49285,"[N11390, N49285, N38951, N62365, N28682, N5472, N26572, N42844, N31958, N12446, N6950, N11930, N...","[N17266, N63062, N46846, N51891, N4912, N37663, N42992, N64273, N32830, N64536]"
3,14,U47134,"[N49554, N12446, N7342, N6638, N12320, N10423, N5940, N60747, N2852, N61829, N37204, N50894, N37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N36940,"[N35809, N13395, N23284, N64496, N29177, N1150, N26136, N37812, N10629, N10059, N27676, N10343, ...",[N36940],56,124,N49554,"[N36940, N49554, N12446, N7342, N6638, N12320, N10423, N5940, N60747, N2852, N61829, N37204, N50...","[N9056, N15246, N65200, N38105, N17246, N50, N53901, N14522, N1774, N4622]"
4,16,U18529,"[N16118, N28072, N37204, N6645, N57515, N15052, N15817, N60939, N29490, N19643, N32536, N5981, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N13854,"[N28850, N17770, N33458, N13724, N12760, N51396, N53033, N501, N64425, N64124, N56148, N16715, N...","[N13854, N5507]",17,117,N16118,"[N13854, N16118, N28072, N37204, N6645, N57515, N15052, N15817, N60939, N29490, N19643, N32536, ...","[N501, N64425, N64124, N56148, N16715, N19028, N30119, N9103, N2994, N36253]"
...,...,...,...,...,...,...,...,...,...,...,...,...
24185,73142,U37385,"[N38915, N62992, N13347, N10051, N61197, N48487, N32567, N54752, N38311, N7419, N19643, N13408, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N44249,"[N61471, N3046, N59173, N19048, N42526, N9226, N9120, N29898, N5102, N61708, N306, N64273, N14006]",[N44249],13,160,N38915,"[N44249, N38915, N62992, N13347, N10051, N61197, N48487, N32567, N54752, N38311, N7419, N19643, ...","[N19048, N42526, N9226, N9120, N29898, N5102, N61708, N306, N64273, N14006]"
24186,73145,U92886,"[N59933, N1952, N60724, N55036, N44621, N21171, N56969, N48740, N19990, N6950, N7556, N11390, N2...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N1952,"[N28741, N35450, N31801, N8201, N14617, N5742, N18656, N64531, N29135, N34862, N34040, N306, N12...","[N1952, N6400, N45319]",24,40,N59933,"[N1952, N59933, N60724, N55036, N44621, N21171, N56969, N48740, N19990, N6950, N7556, N11390, N2...","[N56828, N51330, N8549, N42128, N18766, N4151, N4607, N56527, N50340, N61274]"
24187,73146,U23543,"[N59602, N19990, N53572,

In [14]:
MIND_dev_df_fliter_new = MIND_dev_df_fliter[:10000].reset_index(drop=True)

In [15]:
test_user_set = set(MIND_dev_df_fliter_new["userid"].values.tolist())
test_item_set = set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist())) | set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_history"].values.tolist()))
len(set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist())))
len(test_user_set)
len(test_item_set)

3013

8915

13381

# 3. LabelEncoder

In [16]:
from sklearn import preprocessing

user_le = preprocessing.LabelEncoder()
user_le.fit(list(test_user_set))
print("user id unique nums:", len(user_le.classes_))

item_le = preprocessing.LabelEncoder()
item_le.fit(list(test_item_set))
print("item id unique nums:", len(item_le.classes_))

LabelEncoder()

user id unique nums: 8915


LabelEncoder()

item id unique nums: 13381


In [17]:
item_pop = item_pop[item_pop["itemid"].isin(list(set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist()))))].reset_index(drop=True)
item_pop["itemid"] = pd.Series(item_le.transform(item_pop['itemid']))
item_pop

,itemid,pop
0,5087,8042
1,8450,5093
2,6171,4688
3,11288,4191
4,2385,3826
...,...,...
1988,9863,1
1989,3095,1
1990,6128,1
1991,10914,1


In [18]:
MIND_dev_df_fliter_new["userid"] = pd.Series(user_le.transform(MIND_dev_df_fliter_new['userid']))
MIND_dev_df_fliter_new["full_candidate"] = MIND_dev_df_fliter_new["full_candidate"].apply(lambda x: list(item_le.transform(x)))
MIND_dev_df_fliter_new["full_history"] = MIND_dev_df_fliter_new["full_history"].apply(lambda x: list(item_le.transform(x)))
MIND_dev_df_fliter_new["pos_id_target"] = pd.Series(item_le.transform(MIND_dev_df_fliter_new['pos_target']))
MIND_dev_df_fliter_new["neg_id_target"] = pd.Series(item_le.transform(MIND_dev_df_fliter_new['neg_target']))

MIND_dev_df_fliter_new
MIND_dev_df_fliter_new.to_csv(f"../{dataset_name}/MIND_dev_df_fliter.csv", sep="\t", index=False)

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len,neg_target,full_candidate,full_history,pos_id_target,neg_id_target
0,4,8182,"[N6950, N60215, N6074, N11930, N6916, N24802, N48740, N60675, N45057, N51470, N62365, N15347, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N15347,"[N63554, N49153, N28678, N43369, N58518, N44402, N7649, N63429, N45794, N53531, N53033, N30765, ...",[N15347],17,26,N6950,"[1299, 12733, 11471, 11606, 445, 12726, 3439, 8871, 11586, 8015, 9481, 11982, 2818, 8981, 4404, ...","[12213, 8158, 9977, 9854, 4799, 5650, 3337, 4462, 3690, 4208]",1299,12733
1,11,1531,"[N54125, N60799, N29862, N3159, N46749, N40982, N37793, N31958, N6400, N1952, N38620, N7419, N10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N24802,"[N10792, N63241, N26424, N49745, N46978, N6233, N61103, N12907, N10865, N2783, N55743, N63554, N...","[N24802, N15347]",23,91,N54125,"[3439, 10105, 11619, 4578, 4991, 8378, 7110, 6393, 5087, 12348, 2247, 6577, 12834, 21, 7633, 234...","[7482, 6668, 3094, 2113, 8603, 10503, 2471, 11443, 8873, 8399]",3439,10105
2,12,4264,"[N49285, N38951, N62365, N28682, N5472, N26572, N42844, N31958, N12446, N6950, N11390, N11930, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N11390,"[N30974, N7171, N22561, N51630, N55951, N12098, N33922, N45794, N17559, N36636, N49792, N52551, ...","[N11390, N51470, N24802]",35,37,N49285,"[314, 8981, 6662, 11982, 4301, 10247, 3837, 7533, 5087, 565, 12733, 445, 12861, 12348, 1299, 127...","[1736, 12137, 8400, 9576, 8950, 6364, 7568, 12409, 5293, 12473]",314,8981
3,14,3921,"[N49554, N12446, N7342, N6638, N12320, N10423, N5940, N60747, N2852, N61829, N37204, N50894, N37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N36940,"[N35809, N13395, N23284, N64496, N29177, N1150, N26136, N37812, N10629, N10059, N27676, N10343, ...",[N36940],56,124,N49554,"[6208, 9044, 565, 12817, 12659, 531, 89, 11288, 11607, 4267, 11863, 6269, 9350, 6273, 6174, 9381...","[13196, 1276, 12632, 6452, 1732, 9141, 10060, 1082, 1844, 8260]",6208,9044
4,16,866,"[N16118, N28072, N37204, N6645, N57515, N15052, N15817, N60939, N29490, N19643, N32536, N5981, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N13854,"[N28850, N17770, N33458, N13724, N12760, N51396, N53033, N501, N64425, N64124, N56148, N16715, N...","[N13854, N5507]",17,117,N16118,"[909, 1471, 4171, 6269, 12661, 10881, 1232, 1420, 11656, 4493, 2278, 5210, 11382, 5017, 1992, 40...","[9175, 12441, 12372, 10563, 1607, 2143, 4635, 13203, 4594, 6045]",909,1471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30196,4954,"[N20036, N47383, N6916, N58098, N31958, N12253, N13408, N19611, N36779, N30290, N42844, N5472, N...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N47383,"[N44251, N52946, N55189, N4020, N56447, N50032, N16887, N18030, N15761, N55743, N10053, N31165, ...",[N47383],19,25,N20036,"[8536, 2385, 12726, 10998, 5087, 513, 810, 2271, 6171, 4683, 7533, 10247, 8437, 12618, 2371, 630...","[10476, 13, 4899, 1409, 7482, 9749, 2101, 8873, 10467, 10921]",8536,2385
9996,30200,2395,"[N60675, N60215, N62365, N63860, N27057, N4442, N41099, N28640, N60762, N29091, N6916, N512, N59...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N496,"[N49519, N43092, N21241, N22279, N21777, N54516, N33247, N64503, N36699, N642, N47525, N31983, N...",[N496],38,26,N60675,"[9052, 11586, 11471, 11982, 12315, 3948, 7893, 7138, 4291, 11610, 4404, 12726, 9422, 11314, 1286...","[4174, 931, 1889, 6343, 12019, 5273, 9672, 10208, 8391, 4684]",9052,11586
9997,30205,1723,"[N6916, N20187, N58098, N11556, N44289, N48487, N6638, N36931, N42844, N23855, N36786, N6074, N3...","[

# 4. Gnerate Test Data for Top-K Ranking

In [19]:
def get_sub_history_and_candidate(row, n_candidate=10, n_history=10):
    cand_cache = list(copy.deepcopy(row.full_candidate))
    cand_cache.remove(row.pos_id_target)
    cand_cache.remove(row.neg_id_target)
    candidate = [int(row.pos_id_target)] + [int(row.neg_id_target)] + cand_cache[:n_candidate-2]
    random.shuffle(candidate)
    pos_target_index = candidate.index(row.pos_id_target)
    history = row.full_history[-n_history : ]
    
    return candidate, history, pos_target_index

def get_pairwise_data(row):
    candidate = row.itemid_candidate
    pos = candidate[row.pos_target_index]
    neg_list = copy.deepcopy(candidate)
    neg_list.remove(pos)
    pair_list = []
    answer_list = []
    for neg in neg_list:
        pair = [pos, neg]
        random.shuffle(pair)
        answer = pair.index(pos)
        pair_list.append(pair)
        answer_list.append(answer)
    
    return pair_list, answer_list

def get_topk_final_data(df):
    for n_candidate in [20]:
        for n_history in [5]:
            print(f"n_candidate:{n_candidate} ; n_history{n_history}")
            LLM_top1_data = copy.deepcopy(df)
            random.seed(2023)
            LLM_top1_data[["itemid_candidate", "itemid_history", "pos_target_index"]] = LLM_top1_data.apply(get_sub_history_and_candidate, n_candidate=n_candidate, n_history=n_history, axis=1, result_type="expand")
            LLM_top1_data = LLM_top1_data.sample(frac=1.0, random_state=2023).reset_index(drop=True)
            LLM_top1_data[["pair_itemid_candidate", "pair_pos_target_index"]] = LLM_top1_data.apply(get_pairwise_data, axis=1, result_type="expand")
            LLM_top1_data.to_csv(f"../{dataset_name}/LLM/topk_candidate@{n_candidate}_history@{n_history}.csv", sep="\t", index=False)

In [20]:
import random
get_topk_final_data(MIND_dev_df_fliter_new)

n_candidate:20 ; n_history5


# 5. Generate Datamaps

In [19]:
MIND_dev_news_df_fliter = MIND_dev_news_df_fliter[MIND_dev_news_df_fliter["newsid"].isin(test_item_set)].reset_index(drop=True)
MIND_dev_news_df_fliter
(MIND_dev_news_df_fliter.isna().sum()/MIND_dev_news_df_fliter.shape[0]).sort_values(ascending=False)

,newsid,category,subcategory,title,abstract
0,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...
1,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
2,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge..."
3,N9721,health,nutrition,"50 Foods You Should Never Eat, According to Health Experts",This is so depressing.
4,N18680,health,health-news,"Michigan apple recall: Nearly 2,300 crates could be contaminated with listeria","A Michigan produce company has recalled nearly 2,300 cases of fresh apples that could be contami..."
...,...,...,...,...,...
13376,N16016,health,healthnews,"More than 130,000 pounds of ground beef recalled for possible plastic contamination","About 130,464 pounds of raw ground beef products have been recalled because they may contain pla..."
13377,N7618,autos,autosnews,Ford v Ferrari: the real story,The film about the epic Le Mans rivalry promises to be a must for car fans - and this is the rea...
13378,N16804,music,music-celebrity,Neil Young Says U.S. Citizenship Application Delayed By Marijuana Use,Neil Young has revealed that his attempts to attain U.S. citizenship prior to the 2020 president...
13379,N19926,lifestyle,lifestylebeauty,These Haircuts Are Going to be Huge in 2020,"Bring on the bobs, lobs, and bangs! It's time to embrace a new hair trend or two. Here are 15 po..."


newsid         0.0
category       0.0
subcategory    0.0
title          0.0
abstract       0.0
dtype: float64

In [20]:
MIND_dev_news_df_fliter["itemid"] = pd.Series(item_le.transform(MIND_dev_news_df_fliter['newsid']))
MIND_dev_news_df_fliter

,newsid,category,subcategory,title,abstract,itemid
0,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...,11864
1,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me.",9974
2,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge...",6509
3,N9721,health,nutrition,"50 Foods You Should Never Eat, According to Health Experts",This is so depressing.,13316
4,N18680,health,health-news,"Michigan apple recall: Nearly 2,300 crates could be contaminated with listeria","A Michigan produce company has recalled nearly 2,300 cases of fresh apples that could be contami...",2062
...,...,...,...,...,...,...
13376,N16016,health,healthnews,"More than 130,000 pounds of ground beef recalled for possible plastic contamination","About 130,464 pounds of raw ground beef products have been recalled because they may contain pla...",1454
13377,N7618,autos,autosnews,Ford v Ferrari: the real story,The film about the epic Le Mans rivalry promises to be a must for car fans - and this is the rea...,12871
13378,N16804,music,music-celebrity,Neil Young Says U.S. Citizenship Application Delayed By Marijuana Use,Neil Young has revealed that his attempts to attain U.S. citizenship prior to the 2020 president...,1622
13379,N19926,lifestyle,lifestylebeauty,These Haircuts Are Going to be Huge in 2020,"Bring on the bobs, lobs, and bangs! It's time to embrace a new hair trend or two. Here are 15 po...",2354


In [21]:
MIND_dev_news_df_fliter.to_csv(f"../{dataset_name}/full_item.csv", sep="\t", index=False)

In [22]:
def process_name(s):
    s = s.replace("&amp;", "&")
    s = s.replace("  ", " ")
    s = s.replace("\n", " ")
    s = s.replace("\\", " ")
    return s
MIND_dev_news_df_fliter["title"] = MIND_dev_news_df_fliter["title"].apply(lambda x: process_name(x))
MIND_dev_news_df_fliter["abstract"] = MIND_dev_news_df_fliter["abstract"].apply(lambda x: process_name(x))

id2item_dict = MIND_dev_news_df_fliter.set_index("itemid")["title"].to_dict()
id2item_dict
item2id_dict = MIND_dev_news_df_fliter.set_index("title")["itemid"].to_dict()
item2id_dict

datamaps = {}
datamaps["id2item_dict"] = id2item_dict
datamaps["item2id_dict"] = item2id_dict

import json
json_str = json.dumps(datamaps)
with open(f"../{dataset_name}/LLM/title_datamaps.json", 'w') as out:
    out.write(json_str)
    

{11864: "The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War",
 9974: "I Was An NBA Wife. Here's How It Affected My Mental Health.",
 6509: 'How to Get Rid of Skin Tags, According to a Dermatologist',
 13316: '50 Foods You Should Never Eat, According to Health Experts',
 2062: 'Michigan apple recall: Nearly 2,300 crates could be contaminated with listeria',
 11014: 'Is This The 2021 GMC Yukon Denali?',
 4412: 'John Dorsey admits talks with Washington, but it "takes two to tango"',
 8612: '17 Abandoned Theme Parks to Explore for Thrills, Chills, and Nostalgia',
 12266: '25 Last-Minute Ideas That Will Absolutely Save Your Holiday Dinner',
 11309: 'Every outfit Duchess Kate has worn in 2019',
 11562: 'Best PS4 games 2019: play great PlayStation 4 games',
 6915: 'The 23 Best TV Reboots of All Time',
 11601: 'Extinction Rebellion wins court challenge against London police',
 7716: "POWER RANKING: Here's who has the best chance of becoming the 2020 Democratic presidential nominee

{"The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War": 11864,
 "I Was An NBA Wife. Here's How It Affected My Mental Health.": 9974,
 'How to Get Rid of Skin Tags, According to a Dermatologist': 6509,
 '50 Foods You Should Never Eat, According to Health Experts': 13316,
 'Michigan apple recall: Nearly 2,300 crates could be contaminated with listeria': 2062,
 'Is This The 2021 GMC Yukon Denali?': 11014,
 'John Dorsey admits talks with Washington, but it "takes two to tango"': 4412,
 '17 Abandoned Theme Parks to Explore for Thrills, Chills, and Nostalgia': 8612,
 '25 Last-Minute Ideas That Will Absolutely Save Your Holiday Dinner': 12266,
 'Every outfit Duchess Kate has worn in 2019': 11309,
 'Best PS4 games 2019: play great PlayStation 4 games': 11562,
 'The 23 Best TV Reboots of All Time': 6915,
 'Extinction Rebellion wins court challenge against London police': 11601,
 "POWER RANKING: Here's who has the best chance of becoming the 2020 Democratic presidential nominee": 771

2071911

In [66]:
id2item_dict = MIND_dev_news_df_fliter.set_index("itemid")["abstract"].to_dict()
id2item_dict
item2id_dict = MIND_dev_news_df_fliter.set_index("abstract")["itemid"].to_dict()
item2id_dict

datamaps = {}
datamaps["id2item_dict"] = id2item_dict
datamaps["item2id_dict"] = item2id_dict

import json
json_str = json.dumps(datamaps)
with open(f"./Dataset/{dataset_name}/LLM/abstract_datamaps.json", 'w') as out:
    out.write(json_str)

{11864: 'Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.',
 9974: "I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me.",
 6509: "They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Get Rid of Skin Tags, According to a Dermatologist appeared first on Reader's Digest.",
 13316: 'This is so depressing.',
 2062: 'A Michigan produce company has recalled nearly 2,300 cases of fresh apples that could be contaminated with listeria.',
 11014: 'A Motor1.com reader sent this to us, and it sure looks legit.',
 4412: 'Team officials in Washington "emphatically" denied a rumor of a Trent Williams trade to Cleveland, according to a report Tuesday. A day later, Browns General Manager John Dorsey admitted publicly he has talked to Washington president Bruce Allen. "We

{'Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.': 11864,
 "I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me.": 9974,
 "They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Get Rid of Skin Tags, According to a Dermatologist appeared first on Reader's Digest.": 6509,
 'This is so depressing.': 13316,
 'A Michigan produce company has recalled nearly 2,300 cases of fresh apples that could be contaminated with listeria.': 2062,
 'A Motor1.com reader sent this to us, and it sure looks legit.': 11014,
 'Team officials in Washington "emphatically" denied a rumor of a Trent Williams trade to Cleveland, according to a report Tuesday. A day later, Browns General Manager John Dorsey admitted publicly he has talked to Washington president Bruce Allen. "We\'ve h

FileNotFoundError: [Errno 2] No such file or directory: './Dataset/News/LLM/abstract_datamaps.json'

In [ ]:
item2pop_dict = item_pop.set_index("itemid")["pop"].to_dict()
for i in id2item_dict.keys():
    if i not in item2pop_dict.keys():
        item2pop_dict[i] = 0
item2pop_dict

json_str = json.dumps(item2pop_dict)
with open(f"./Dataset/{dataset_name}/LLM/popularity_datamaps.json", 'w') as out:
    out.write(json_str)

## Example of Loading Final Evaluation Data 

In [ ]:
read_example_df = pd.read_csv(f"./Dataset/{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "itemid_history", "pos_target_index", "itemid_candidate", "pair_itemid_candidate", "pair_pos_target_index"]]
for col in ['itemid_history', 'itemid_candidate', "pair_itemid_candidate", "pair_pos_target_index"]:
    read_example_df[col] = read_example_df[col].apply(lambda x: eval(x))
    
read_example_df

,userid,itemid_history,pos_target_index,itemid_candidate,pair_itemid_candidate,pair_pos_target_index
0,2223,"[315, 4611, 11549, 9355, 6963]",3,"[10805, 5205, 1256, 11960, 8488]","[[11960, 10805], [11960, 5205], [1256, 11960], [11960, 8488]]","[0, 0, 1, 0]"
1,6951,"[12195, 11577, 7086, 89, 3786]",3,"[9766, 7805, 5777, 1374, 1663]","[[1374, 9766], [7805, 1374], [1374, 5777], [1663, 1374]]","[0, 1, 0, 1]"
2,8289,"[12301, 6691, 2020, 5144, 11053]",0,"[11436, 5073, 3589, 11386, 9675]","[[11436, 5073], [11436, 3589], [11436, 11386], [9675, 11436]]","[0, 0, 0, 1]"
3,6217,"[4806, 7064, 468, 8379, 7139]",4,"[9673, 3219, 11257, 8609, 4579]","[[9673, 4579], [4579, 3219], [11257, 4579], [4579, 8609]]","[1, 0, 1, 0]"
4,8188,"[3457, 5503, 11137, 4090, 4098]",4,"[5044, 6230, 3689, 11161, 4739]","[[4739, 5044], [6230, 4739], [4739, 3689], [4739, 11161]]","[0, 1, 0, 0]"
...,...,...,...,...,...,...
9995,6047,"[6392, 2427, 10769, 10951, 10751]",3,"[8704, 8855, 7041, 7981, 11518]","[[7981, 8704], [8855, 7981], [7981, 7041], [11518, 7981]]","[0, 1, 0, 1]"
9996,8165,"[5925, 1366, 5742, 4277, 6735]",3,"[9252, 5777, 10513, 9843, 7654]","[[9843, 9252], [9843, 5777], [9843, 10513], [9843, 7654]]","[0, 0, 0, 0]"
9997,6227,"[6994, 8853, 11844, 6063, 1286]",0,"[4757, 10240, 7886, 10350, 7529]","[[4757, 10240], [7886, 4757], [4757, 10350], [4757, 7529]]","[0, 1, 0, 0]"
9998,4203,"[7771, 5144, 11482, 9193, 12196]",3,"[9641, 11518, 6077, 2445, 11870]","[[9641, 2445], [2445, 11518], [6077, 2445], [2445, 11870]]","[1, 0, 1, 0]"


In [ ]:
read_example_df = pd.read_csv(f"./Dataset/{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "pos_target_index", "itemid_candidate"]]
read_example_df["itemid_candidate"] = read_example_df["itemid_candidate"].apply(lambda x: x[1:-1])
candidate = read_example_df['itemid_candidate'].str.split(',', expand=True)
for col in candidate.columns:
    candidate[col] = candidate[col].apply(lambda x: int(x))
candidate.values

array([[10805,  5205,  1256, 11960,  8488],
       [ 9766,  7805,  5777,  1374,  1663],
       [11436,  5073,  3589, 11386,  9675],
       ...,
       [ 4757, 10240,  7886, 10350,  7529],
       [ 9641, 11518,  6077,  2445, 11870],
       [10476,  3948,  5447, 10342,  2968]])